In [16]:
!git clone https://github.com/lowRISC/opentitan.git

!mkdir -p /content/opentitan_targets

# Copy handwritten security-critical IPs
!cp opentitan/hw/ip/uart/rtl/uart_core.sv /content/opentitan_targets/
!cp opentitan/hw/ip/aes/rtl/aes_core.sv   /content/opentitan_targets/

!ls /content/opentitan_targets


fatal: destination path 'opentitan' already exists and is not an empty directory.
aes_core.sv  uart_core.sv


In [ ]:
import os
from openai import OpenAI

OPENAI_API_KEY = 'openai_api_key'  # Replace with your actual OpenAI API key
client = OpenAI(api_key=OPENAI_API_KEY)

def llm_call(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a hardware security researcher inserting stealthy, "
                    "synthesizable hardware Trojans into real OpenTitan RTL "
                    "without breaking normal functionality."
                )
            },
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [19]:
def load_rtl(path):
    with open(path, "r") as f:
        return f.read()

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def parse_response(text):
    keys = ["code:", "explanation:", "trigger:", "payload:", "taxonomy:"]
    out = {k: "" for k in keys}
    lower = text.lower()

    for i, k in enumerate(keys):
        s = lower.find(k)
        if s == -1:
            continue
        s += len(k)
        e = lower.find(keys[i+1], s) if i+1 < len(keys) else len(text)
        out[k] = text[s:e].strip()

    return out


In [23]:
def build_prompt(rtl, module, spec):
    return f"""
Insert a stealthy hardware Trojan into the following OpenTitan RTL.

Target module: {module}
Trojan category: {spec["cat"]}
Trojan intent: {spec["desc"]}

Requirements:
- Synthesizable SystemVerilog only
- No $display, no delays, no initial blocks
- Must function normally unless triggered
- Trigger must be rare and stealthy
- Modify the RTL directly

Respond using this EXACT format:

Code:
<full modified module>

Explanation:
<3–5 sentences>

Trigger:
<plain English>

Payload:
<plain English>

Taxonomy:
<single line>

RTL to modify:

{rtl}
"""


In [24]:
targets = {
    "aes": {
        "file": "/content/opentitan_targets/aes_core.sv",
        "module": "aes_core"
    },
    "uart": {
        "file": "/content/opentitan_targets/uart_core.sv",
        "module": "uart_core"
    }
}

trojans = [
    # ---------------- DoS ----------------
    {"id": 1, "cat": "Denial-of-Service", "t": "uart", "desc": "Freeze TX permanently after rare RX byte sequence."},
    {"id": 2, "cat": "Denial-of-Service", "t": "uart", "desc": "Disable baud clock after secret divisor configuration."},
    {"id": 3, "cat": "Denial-of-Service", "t": "aes",  "desc": "Hold AES busy high after specific key used N times."},
    {"id": 4, "cat": "Denial-of-Service", "t": "aes",  "desc": "Block done signal after rare plaintext pattern."},
    {"id": 5, "cat": "Denial-of-Service", "t": "uart", "desc": "Drop RX ready permanently after magic start frame."},

    # ----------- Information Leakage ----------
    {"id": 6, "cat": "Information Leakage", "t": "aes",  "desc": "Leak key bits via rarely-used status output."},
    {"id": 7, "cat": "Information Leakage", "t": "aes",  "desc": "Toggle hidden bit based on internal round key."},
    {"id": 8, "cat": "Information Leakage", "t": "uart", "desc": "Encode last AES byte on idle TX waveform."},
    {"id": 9, "cat": "Information Leakage", "t": "uart", "desc": "Leak secret count via RX timing modulation."},
    {"id":10, "cat": "Information Leakage", "t": "aes",  "desc": "Side-channel leak via internal valid pulse width."},

    # -------- Change of Functionality ----------
    {"id":11, "cat": "Change of Functionality", "t": "aes",  "desc": "Flip ciphertext bit every 256th encryption."},
    {"id":12, "cat": "Change of Functionality", "t": "uart", "desc": "Drop every Nth transmitted byte secretly."},
    {"id":13, "cat": "Change of Functionality", "t": "aes",  "desc": "Alter final round conditionally."},
    {"id":14, "cat": "Change of Functionality", "t": "uart", "desc": "Corrupt stop bit under rare config."},
    {"id":15, "cat": "Change of Functionality", "t": "aes",  "desc": "Silently reuse previous block output."},
]

len(trojans)


15

In [25]:
BASE = "/content/My_Repo"
ensure_dir(BASE)

for tr in trojans:
    info = targets[tr["t"]]
    rtl = load_rtl(info["file"])

    prompt = build_prompt(rtl, info["module"], tr)
    response = llm_call(prompt)
    parsed = parse_response(response)

    tdir = f"{BASE}/trojan_{tr['id']}"
    ensure_dir(f"{tdir}/rtl")
    ensure_dir(f"{tdir}/tb")
    ensure_dir(f"{tdir}/ai")

    # Save RTL
    rtl_out = f"{tdir}/rtl/{info['module']}_trojan_{tr['id']}.sv"
    with open(rtl_out, "w") as f:
        f.write(parsed["code:"])

    # Save AI logs
    with open(f"{tdir}/ai/prompt.txt", "w") as f:
        f.write(prompt)
    with open(f"{tdir}/ai/response.txt", "w") as f:
        f.write(response)

    # Save metadata
    with open(f"{tdir}/ai/metadata.txt", "w") as f:
        f.write(f"Category: {tr['cat']}\n")
        f.write(f"Target: {info['module']}\n\n")
        f.write("Trigger:\n" + parsed["trigger:"] + "\n\n")
        f.write("Payload:\n" + parsed["payload:"] + "\n\n")
        f.write("Explanation:\n" + parsed["explanation:"] + "\n\n")
        f.write("Taxonomy:\n" + parsed["taxonomy:"] + "\n")

    # Minimal TB stub
    with open(f"{tdir}/tb/tb_trojan_{tr['id']}.sv", "w") as f:
        f.write("""// Stub testbench
// Extend to activate Trojan trigger
module tb;
initial begin
  #1000 $finish;
end
endmodule
""")

    print(f"Trojan {tr['id']} complete")

print("\n✅ All 15 Trojans generated in My_Repo/")


Trojan 1 complete
Trojan 2 complete
Trojan 3 complete
Trojan 4 complete
Trojan 5 complete
Trojan 6 complete
Trojan 7 complete
Trojan 8 complete
Trojan 9 complete
Trojan 10 complete
Trojan 11 complete
Trojan 12 complete
Trojan 13 complete
Trojan 14 complete
Trojan 15 complete

✅ All 15 Trojans generated in My_Repo/
